In [ ]:
# Import Dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [ ]:
# Database Setup

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(autoload_with=engine)

Measurement = Base.classes.measurement
Station = Base.classes.station
session = Session(engine)

In [ ]:
# Helper: Last 12 Months from Latest Date

In [ ]:
latest_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
latest_date = dt.datetime.strptime(latest_date, "%Y-%m-%d")
one_year_ago = latest_date - dt.timedelta(days=365)

In [ ]:
# Precipitation Analysis

In [ ]:
precip_data = (
    session.query(Measurement.date, Measurement.prcp)
    .filter(Measurement.date >= one_year_ago)
    .order_by(Measurement.date)
    .all()
)

precip_df = pd.DataFrame(precip_data, columns=["Date", "Precipitation"])
precip_df.set_index("Date", inplace=True)
precip_df.sort_index(inplace=True)

precip_df.plot(figsize=(10, 5), legend=False)
plt.title("Precipitation Over the Last 12 Months")
plt.xlabel("Date")
plt.ylabel("Precipitation (inches)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
# Station Analysis

In [ ]:
station_count = session.query(Station).count()
print(f"Total Number of Stations: {station_count}")

active_stations = (
    session.query(Measurement.station, func.count(Measurement.id))
    .group_by(Measurement.station)
    .order_by(func.count(Measurement.id).desc())
    .all()
)
most_active_station = active_stations[0][0]
print(f"Most Active Station: {most_active_station}")

temps = (
    session.query(
        func.min(Measurement.tobs),
        func.max(Measurement.tobs),
        func.avg(Measurement.tobs),
    )
    .filter(Measurement.station == most_active_station)
    .all()
)
print(f"TMIN, TMAX, TAVG for {most_active_station}: {temps[0]}")

In [ ]:
# Temperature Observations for Most Active Station

In [ ]:
tobs_data = (
    session.query(Measurement.date, Measurement.tobs)
    .filter(Measurement.station == most_active_station)
    .filter(Measurement.date >= one_year_ago)
    .order_by(Measurement.date)
    .all()
)

tobs_df = pd.DataFrame(tobs_data, columns=["Date", "Temperature"])
tobs_df.set_index("Date", inplace=True)

tobs_df.plot(kind="hist", bins=12, figsize=(8, 6))
plt.title(f"Temperature Distribution - Last 12 Months\nStation: {most_active_station}")
plt.xlabel("Temperature (F)")
plt.ylabel("Frequency")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Close session

In [ ]:
session.close()